# Прогнозирование временных рядов с использованием Informer

Автоматический поиск архитектур (NAS) для задачи time-series forecasting с метрикой MSE.

## 1. Импорт библиотек

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import optuna
import warnings
warnings.filterwarnings('ignore')

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(f"Устройство: {device}")

## 2. Загрузка и подготовка данных

In [ ]:
url_eth1 = "https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTh1.csv"
url_eth2 = "https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTh2.csv"

df_eth1 = pd.read_csv(url_eth1)
df_eth2 = pd.read_csv(url_eth2)

print(f"ETTh1: {df_eth1.shape}")
print(f"ETTh2: {df_eth2.shape}")
print(f"Столбцы: {df_eth1.columns.tolist()}")

In [ ]:
train_size = 12 * 30 * 24
val_size = 4 * 30 * 24

train_df = df_eth1[:train_size]
val_df = df_eth1[train_size:train_size + val_size]
test_df = df_eth1[train_size + val_size:train_size + 2*val_size]

feature_cols = [col for col in df_eth1.columns if col != 'date']
target_col = 'OT'

print(f"Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")
print(f"Признаки: {feature_cols}")

## 3. Класс Dataset

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len, label_len, pred_len, features, target='OT', flag='train', scale=True, scaler=None):
        self.seq_len = seq_len
        self.label_len = label_len
        self.pred_len = pred_len
        self.features = features
        self.target = target
        self.flag = flag
        
        self.data_x = data[features].values
        self.data_y = data[target].values
        
        if scale:
            if scaler is None:
                self.scaler = StandardScaler()
                self.data_x = self.scaler.fit_transform(self.data_x)
            else:
                self.scaler = scaler
                self.data_x = self.scaler.transform(self.data_x)
        else:
            self.scaler = None
    
    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1
    
    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len
        
        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_x[r_begin:r_end]
        seq_x_mark = np.zeros((self.seq_len, 4))
        seq_y_mark = np.zeros((self.label_len + self.pred_len, 4))
        
        return seq_x, seq_y, seq_x_mark, seq_y_mark



## 4. Клонирование репозитория Informer2020

In [ ]:
if not os.path.exists('Informer2020'):
    !git clone https://github.com/zhouhaoyi/Informer2020.git
    print("Репозиторий клонирован")
else:
    print("Репозиторий уже существует")

sys.path.append('Informer2020')
from models.model import Informer

## 5. Функция создания модели

In [ ]:
def create_model(config):
    model = Informer(
        enc_in=config['enc_in'],
        dec_in=config['dec_in'],
        c_out=config['c_out'],
        seq_len=config['seq_len'],
        label_len=config['label_len'],
        out_len=config['pred_len'],
        factor=config.get('factor', 5),
        d_model=config['d_model'],
        n_heads=config['n_heads'],
        e_layers=config['e_layers'],
        d_layers=config['d_layers'],
        d_ff=config['d_ff'],
        dropout=config['dropout'],
        attn='prob',
        embed='timeF',
        freq='h',
        activation='gelu',
        output_attention=False,
        distil=True,
        mix=True
    ).to(device)
    return model


## 6. Цикл обучения

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, label_len, pred_len):
    model.train()
    total_loss = 0
    
    for batch_x, batch_y, batch_x_mark, batch_y_mark in dataloader:
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        
        dec_inp = torch.zeros_like(batch_y[:, -pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :label_len, :], dec_inp], dim=1).float().to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        
        f_dim = -1
        outputs = outputs[:, -pred_len:, f_dim:]
        batch_y = batch_y[:, -pred_len:, f_dim:].to(device)
        
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

def validate(model, dataloader, criterion, label_len, pred_len):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch_x, batch_y, batch_x_mark, batch_y_mark in dataloader:
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            batch_x_mark = batch_x_mark.float().to(device)
            batch_y_mark = batch_y_mark.float().to(device)
            
            dec_inp = torch.zeros_like(batch_y[:, -pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :label_len, :], dec_inp], dim=1).float().to(device)
            
            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
            
            f_dim = -1
            outputs = outputs[:, -pred_len:, f_dim:]
            batch_y = batch_y[:, -pred_len:, f_dim:].to(device)
            
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    
    return total_loss / len(dataloader)


## 7. Optuna: Подбор гиперпараметров

In [ ]:
def objective(trial):
    d_model = trial.suggest_categorical('d_model', [128, 256, 512])
    
    if d_model == 128:
        n_heads = trial.suggest_categorical('n_heads', [4, 8])
    elif d_model == 256:
        n_heads = trial.suggest_categorical('n_heads', [4, 8])
    else:  # 512
        n_heads = trial.suggest_categorical('n_heads', [8, 16])
    
    config = {
        'enc_in': len(feature_cols),
        'dec_in': len(feature_cols),
        'c_out': len(feature_cols),
        'seq_len': trial.suggest_categorical('seq_len', [168, 336, 720]),
        'label_len': trial.suggest_categorical('label_len', [24, 48, 72]),
        'pred_len': 96,  
        'd_model': d_model,
        'n_heads': n_heads,
        'e_layers': trial.suggest_int('e_layers', 2, 4),
        'd_layers': trial.suggest_int('d_layers', 1, 2),
        'd_ff': trial.suggest_categorical('d_ff', [d_model * 2, d_model * 4]),
        'dropout': trial.suggest_float('dropout', 0.0, 0.5),
        'factor': trial.suggest_int('factor', 3, 5),
    }
    
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    
    train_dataset = TimeSeriesDataset(
        train_df, config['seq_len'], config['label_len'], config['pred_len'], 
        feature_cols, target_col, 'train', scale=True
    )
    val_dataset = TimeSeriesDataset(
        val_df, config['seq_len'], config['label_len'], config['pred_len'],
        feature_cols, target_col, 'val', scale=True, scaler=train_dataset.scaler
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    model = create_model(config)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    
    best_val_loss = float('inf')
    patience = 5  
    patience_counter = 0
    
    for epoch in range(20): 
        train_loss = train_epoch(model, train_loader, criterion, optimizer, config['label_len'], config['pred_len'])
        val_loss = validate(model, val_loader, criterion, config['label_len'], config['pred_len'])
        scheduler.step()
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            break
        
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return best_val_loss


In [ ]:
sampler = optuna.samplers.TPESampler(
    seed=777, 
    n_startup_trials=10, 
    multivariate=True,  
)

pruner = optuna.pruners.MedianPruner(
    n_startup_trials=5,  
    n_warmup_steps=5,  
    interval_steps=1,  
)

study = optuna.create_study(
    direction='minimize',
    study_name='informer_nas_improved',
    sampler=sampler,
    pruner=pruner,
)

study.optimize(objective, n_trials=50, show_progress_bar=True)


## 8. Обучение финальной модели

In [ ]:
best_config = {
    'enc_in': len(feature_cols),
    'dec_in': len(feature_cols),
    'c_out': len(feature_cols),
    'seq_len': study.best_params['seq_len'],
    'label_len': study.best_params['label_len'],
    'pred_len': 96,
    'd_model': study.best_params['d_model'],
    'n_heads': study.best_params['n_heads'],
    'e_layers': study.best_params['e_layers'],
    'd_layers': study.best_params['d_layers'],
    'd_ff': study.best_params['d_ff'],
    'dropout': study.best_params['dropout'],
    'factor': study.best_params.get('factor', 5),  # factor из оптимизации
}

best_batch_size = study.best_params.get('batch_size', 32)
best_lr = study.best_params.get('learning_rate', 1e-4)

train_dataset = TimeSeriesDataset(
    train_df, best_config['seq_len'], best_config['label_len'], best_config['pred_len'],
    feature_cols, target_col, 'train', scale=True
)
val_dataset = TimeSeriesDataset(
    val_df, best_config['seq_len'], best_config['label_len'], best_config['pred_len'],
    feature_cols, target_col, 'val', scale=True, scaler=train_dataset.scaler
)
test_dataset = TimeSeriesDataset(
    test_df, best_config['seq_len'], best_config['label_len'], best_config['pred_len'],
    feature_cols, target_col, 'test', scale=True, scaler=train_dataset.scaler
)

train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False, num_workers=0)

model = create_model(best_config)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=best_lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

print(f"Параметров: {sum(p.numel() for p in model.parameters()):,}")
print(f"Batch size: {best_batch_size}")
print(f"Learning rate: {best_lr:.2e}")

In [ ]:
epochs = 50
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, best_config['label_len'], best_config['pred_len'])
    val_loss = validate(model, val_loader, criterion, best_config['label_len'], best_config['pred_len'])
    scheduler.step()  
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
    
    if (epoch + 1) % 5 == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, LR: {current_lr:.2e}")

print(f"\nОбучение завершено. Лучший Val Loss: {best_val_loss:.6f}")

## 9. Оценка на тестовых данных

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))
test_loss = validate(model, test_loader, criterion, best_config['label_len'], best_config['pred_len'])

print(f"Test MSE: {test_loss:.6f}")
print(f"Test RMSE: {np.sqrt(test_loss):.6f}")

## 10. Визуализация

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Train Loss', linewidth=2)
plt.plot(val_losses, label='Val Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Процесс обучения')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
model.eval()
predictions = []
actuals = []

label_len = best_config['label_len']
pred_len = best_config['pred_len']

with torch.no_grad():
    for batch_x, batch_y, batch_x_mark, batch_y_mark in test_loader:
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        
        dec_inp = torch.zeros_like(batch_y[:, -pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :label_len, :], dec_inp], dim=1).float().to(device)
        
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        
        f_dim = -1
        outputs = outputs[:, -pred_len:, f_dim:]
        batch_y = batch_y[:, -pred_len:, f_dim:]
        
        predictions.append(outputs.cpu().numpy())
        actuals.append(batch_y.cpu().numpy())

predictions = np.concatenate(predictions, axis=0)
actuals = np.concatenate(actuals, axis=0)

target_idx = feature_cols.index(target_col)
pred_target = predictions[:, :, target_idx].flatten()[:500]
actual_target = actuals[:, :, target_idx].flatten()[:500]

plt.figure(figsize=(15, 6))
plt.plot(actual_target, label='Реальные значения', linewidth=2, color='green')
plt.plot(pred_target, label='Прогноз', linewidth=2, color='red', alpha=0.8)
plt.xlabel('Временной шаг')
plt.ylabel(f'{target_col} (Temperature)')
plt.title(f'Прогноз vs Реальные значения - {target_col}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

mse = mean_squared_error(actual_target, pred_target)
mae = mean_absolute_error(actual_target, pred_target)
print(f"\nМетрики на тестовой выборке:")
print(f"MSE: {mse:.6f}")
print(f"RMSE: {np.sqrt(mse):.6f}")
print(f"MAE: {mae:.6f}")

## 11. История Optuna

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

fig = optuna.visualization.plot_param_importances(study)
fig.show()